In [1]:
import coiled
import dask.distributed
import dask.dataframe as dd

In [2]:
cluster = coiled.Cluster(configuration="coiled/default", n_workers=5)

Output()

Found software environment build


In [3]:
client = dask.distributed.Client(cluster)

In [4]:
d2000 = dd.read_parquet("s3://coiled-datasets/timeseries/7d/parquet/2000")
d2001 = dd.read_parquet("s3://coiled-datasets/timeseries/7d/parquet/2001")
d2002 = dd.read_parquet("s3://coiled-datasets/timeseries/7d/parquet/2002")
d2003 = dd.read_parquet("s3://coiled-datasets/timeseries/7d/parquet/2003")
d2004 = dd.read_parquet("s3://coiled-datasets/timeseries/7d/parquet/2004")

In [5]:
%%time
sorted2000 = d2000.set_index("id").head(10)

CPU times: user 305 ms, sys: 79.4 ms, total: 384 ms
Wall time: 17.6 s


In [6]:
sorted2000

,name,x,y
id,,,
832,Oliver,-0.015893,0.678568
842,Yvonne,-0.690717,0.110788
843,Yvonne,-0.873540,-0.270038
843,Oliver,0.514363,0.803397
844,Yvonne,0.664280,-0.507049
844,Hannah,-0.316901,-0.002933
845,Laura,0.569720,-0.311269
846,Victor,0.397965,0.083829
846,Hannah,0.988103,0.959610


In [7]:
ddf = dd.concat([d2000, d2001, d2002, d2003, d2004])

In [8]:
ddf.head()

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1008,Dan,-0.259374,-0.118314
2000-01-01 00:00:01,987,Patricia,0.069601,0.755351
2000-01-01 00:00:02,980,Zelda,-0.281843,-0.510507
2000-01-01 00:00:03,1020,Ursula,-0.569904,0.523132
2000-01-01 00:00:04,967,Michael,-0.251460,0.810930


In [9]:
%%time
sorted_ddf_id = ddf.set_index("id").head(10)

CPU times: user 865 ms, sys: 94 ms, total: 959 ms
Wall time: 1min 1s


In [10]:
sorted_ddf_id

,name,x,y
id,,,
832,Oliver,-0.015893,0.678568
832,Oliver,-0.015893,0.678568
832,Oliver,-0.015893,0.678568
832,Oliver,-0.015893,0.678568
832,Oliver,-0.015893,0.678568
842,Yvonne,-0.690717,0.110788
842,Yvonne,-0.690717,0.110788
842,Yvonne,-0.690717,0.110788
842,Yvonne,-0.690717,0.110788
